<a id="top"></a>
# Open Universe FITS to ASDF File Converter
***
## Learning Goal
By the end of this tutorial, you will:

- Understand how to use the provided file converter to convert the FITS format of Open Universe data into ASDF files
- Have a better understanding of the benefits to using ASDF files versus FITS files

## Introduction

Something should go here...

**THIS TABLE OF CONTENTS NEEDS TO BE UPDATED**
The workflow for this notebook consists of:
* [Main Content](#Main-Content)
    * [Loading Data](#Loading-Data)
    * [File and Data Information](#File-and-Data-Information)
* [Visualization](#Visualization)
* [Exercises](#Exercises)
* [Additional Resources](#Additional-Resources)

## Imports
We have prepared an Open Universe FITS to ASDF Converter (OUFAC) and provided it in the `oufac.py` file. This module primarily contains a `FitsToAsdf` class that performs the conversion. We will also use the `asdf` package to explore the newly converted data after we convert a file.

In [ ]:
import s3fs
from oufac import FitsToAsdf
import asdf
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

The Open Universe data relies on parquet files to store the catalog files which require an additional package to read that is not natively on the science platform. Thus we perform a manual pip install:

In [ ]:
%pip install pyarrow

***

## Converting Data

The first step to converting the data is to determine which file we want to convert. Here we specify the s3 bucket containing the simulated data, choose a band, and a healpix index and use them to construct a single detector image file.

In [ ]:
s3bucket = 's3://nasa-irsa-simulations/openuniverse2024/roman/preview/RomanWAS/images/simple_model'
band = 'F184'
hpix = '9111'
sensor = 2
fits_filename = f'Roman_WAS_simple_model_{band}_{hpix}_{sensor}.fits.gz'
s3fpath = s3bucket+f'/{band}/{hpix}/{fits_filename}'

Now that we think we have the URI to the file we have, it is always good practice to ensure that that path exists using s3fs and the s3fs.ls command to list all files in the path.

In [ ]:
fs = s3fs.S3FileSystem(anon=True)
fs.ls(s3fpath)

If we receive a list with a single string URI inside, we are ready to proceed with the conversion. If you received an empty list or an error, please confirm the file path URI is correct.

### Using the packaged converter

Now that we have confirmed our URI points to the file we want, we are ready to use the packaged converter. 

**NOTE:** the following cell will take roughly 3.5 minutes to run. While it is running, please read on to learn more about the converter.

In [ ]:
asdf_filename = f'Roman_WAS_simple_model_{band}_{hpix}_{sensor}.asdf'
fa = FitsToAsdf(s3fpath)
fa.write(asdf_filename)

While the converter is running, we should expect there to be two types of warnings that are thrown:

The first is a warning that our Simple Imaging Polynomial (SIP) is not sufficient to meet a maximum residual at the current degree. This error is thrown while the converter is gathering the sources from the given catalogs and does not impact the simulated image at all. Rather it has a slight impact on the accuracy of the RA and DEC of the catalog sources within the detector, however the sources are still easy to correlate with the bright sources in the image. The resulting warning should resemble:
```
    Maximum specified SIP approximation error: 5
       - SIP degree: 1. Maximum residual: 0.0091165
    * Maximum residual, double sampled grid: 0.0091165
    * Final SIP degree: 1. Maximum residual: 0.0091165

```

The second has to do with ASDF versioning while the file is saving. The warning is alerting the user that some expected schema documentation does not match. The resulting warnings should resemble:
```
    /opt/conda/envs/roman-cal/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
      warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
    /opt/conda/envs/roman-cal/lib/python3.11/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
      warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
```

Now that we are not worried about the warnings we should expect, we can explain some added functionality to the converter. 

## Exploring the Converted ASDF File

Now that we have created the ASDF file, we can open it using `asdf.open`. The `.info()` method then prints out the overall structure of the ASDF file making it easy to see the contents.

In [ ]:
converted_file = asdf.open(asdf_filename)
converted_file.info(max_rows=40)

The scientifically useful information is stored within the `roman` dictionary which contains all the source catalogs in `catalogs`, all the metadata within `meta`, as well as the image data (`data`), image error (`err`), and data quality (`dq`). The `wcs` key contains a `gwcs.WCS` object that allows for easy access to the WCS information of the simulated data.

Additionally, all the catalogs are stored within `astropy.table.Table`s which allows for a user to easily query them. The galaxy source catalog table is previewed in the below cell. We have truncated the table at 10 elements because the full tables can contain >10k rows.

In [ ]:
converted_file['roman']['catalogs']['galaxies'][:10]

Lastly, we plot the simulated image using `matplotlib` and a logarithmic color normalization to make the sources easier to view.

In [ ]:
plt.imshow(converted_file['roman']['data'], norm=LogNorm())

In [ ]:
plt.imshow(converted_file['roman']['data'][2500:3500, 750:1750], norm=LogNorm())

## Additional Resources

Something should probably go here...

## Citations
Provide your reader with guidelines on how to cite open source software and other resources in their own published work.

```
If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

```

## About this Notebook

**Author(s):** Javier Sanchez, Will C. Schultz <br>
**Keyword(s):** Tutorial, FITS, ASDF, Open-Universe <br>
**Last Updated:** Sep 2024 <br>
**Next Review:** Sep 2024
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 